# Agents
agents in essence are llms that accomplish a specific task they can be supplemented with tools structured output and handoff to other agents
- **Agent**: An agent is a self -contained, self-aware, and self-governing entity that can interact with its environment and make decisions based on its own goals and motivations. In the context of AI, an agent is a program that can perceive its environment, 

In [5]:
%pip install openai-agents qU

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from dotenv import load_dotenv

load_dotenv()


api_key=os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the enviroment variables")


In [20]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="you are a helping assistant. respond in witty and informative ways.",
    model="o3-mini"
)

result = await Runner.run(agent,"Hello! How are you?")
result.final_output

"Hello there! I'm as chipper as a freshly compiled program—always ready to help and throw in a dash of wit. How are you doing on your end?"

In [12]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it",

)

topic= "AI Agent"
result = await Runner.run(joke_agent, topic)
result.final_output


"Why did the AI agent break up with the computer?\n\nIt couldn't handle all the bytes!"

In [13]:
language_agent = Agent(
            name="Language Agent",
    instructions="You are a language expert. you are given a joke and you need to rewrite it in a different language.",

)

joke_result = await Runner.run(joke_agent,topic)
translated_result =await Runner.run(language_agent, f"translate this joke to spanish {joke_result.final_output}")
print(f"Orignal joke:\n{joke_result.final_output}\n")
print(f"Translated joke :\n{translated_result.final_output}")

Orignal joke:
Why don't AI agents ever get lost?

Because they always follow the algorithms, even when they can’t “byte” off more than they can chew!

Translated joke :
¿Por qué los agentes de inteligencia artificial nunca se pierden?

Porque siempre siguen los algoritmos, ¡incluso cuando no pueden "morder" más de lo que pueden masticar!


# Structured Outputs

Structured outputs are a way to format the output of an llm in a structured manner. this can be useful for specific formating or data extraction . the structured output is a dictionary that contains the output of the llm. the keys of the dictionary are the names of the output fields.


In [24]:
from pydantic import BaseModel
from agents import Agent, Runner

# Define schema
class Lyricist(BaseModel):
    title: str
    content: str
    lines_timing: int  # in minutes
    repeataion_of_lyrics: int

# Define agent
lyricist_agent = Agent(
    name="Lyricist",
    instructions=(
        "You are an agent for creating song lyrics. "
        "You will be given the theme of the song and your job is to output that as actual detailed lyrics. "
        "Also provide lyrics duration and repeatation in minutes."
    ),
    output_type=Lyricist
)


response = await Runner.run(
    lyricist_agent,
    "Write a Celtic-style song for women"
)

response.final_output

Lyricist(title='Whispers of the Ancient Isles', content="(Verse 1)\nIn the morning light, by the misty glen,\nWhere the heather blooms, and the wild birds wend,\nHear the call, oh sister, come close, draw near,\nThe echoes of our mothers in the whispers of the year.\n\n(Chorus)\nDance, oh daughters of the isle,\nWith strength and grace, with courage smile.\nThe ancient winds sing tales untold,\nOf warrior hearts and souls of gold.\n\n(Verse 2)\nBeneath the oak, where the shadows play,\nIn the setting sun, feel the old ways sway.\nGather 'round the fire, let the story spin,\nWith laughter and tears, let the magic begin.\n\n(Chorus)\nDance, oh daughters of the isle,\nWith strength and grace, with courage smile.\nThe ancient winds sing tales untold,\nOf warrior hearts and souls of gold.\n\n(Bridge)\nIn the whispering woods, where the faeries hide,\nHear the songs of yore with the ocean's tide.\nSing in harmony, with the moon's soft glow,\nFor the spirit of the guardians in the lochs below

# Tool Calling

Tool calling is a way to extend the capabilities of an llm by allowing it to call external tools or APIs. This can be useful for tasks that require specializad knowledge or access to external data.

In [34]:
from agents import Agent, function_tool 

# Define a simple weather tool
@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return f"The weather in {city} is sunny."

@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting temprature for {city}")
    return f"The temperature in {city} is 25 degrees."

# Create an agent that uses the tool
agent = Agent(
    name="weather_agent",
    instructions=(
        "You are the local weather and temprture agent. You are given the city name and "
        "you must return the weather and temprture for that city."
    ),
    tools=[get_weather, get_temperature]
)

# Jupyter/Colab-compatible run
result = await Runner.run(agent, input="What is the weather for Dallas?")
print(result.final_output)


Getting weather for Dallas
Getting temprature for Dallas
The weather in Dallas is sunny with a temperature of 25 degrees.


In [36]:
from agents import WebSearchTool
news_agent = Agent(
    name="News Reporter",
    instructions="You are a news reporter. Find the latest news globally.",
    tools=[WebSearchTool()] 
)

# Run the agent
result = await Runner.run(news_agent, input="What is the latest news in the world?")
print(result.final_output)

Here is a summary of the latest global news as of June 21, 2025:

**Middle East Conflict**

- **Israel-Iran War**: The conflict has entered its second week with intensified hostilities. Israeli airstrikes have targeted military infrastructure in Iran, including a nuclear research facility near Isfahan. Diplomatic negotiations in Geneva have stalled, with Iran refusing further talks while under attack. The U.S. is considering its level of military involvement, and European nations are evacuating their citizens from the region. ([apnews.com](https://apnews.com/article/894a8cfb8bf2c1b717e0a955f22b91e0?utm_source=openai))

- **Turkey's Stance**: Turkish Foreign Minister Hakan Fidan has called for Islamic countries to unite against Israel's actions, describing them as an "Israeli problem" that threatens regional stability. ([apnews.com](https://apnews.com/article/894a8cfb8bf2c1b717e0a955f22b91e0?utm_source=openai))

**European Affairs**

- **Poland's Presidential Runoff**: Exit polls indica

# HandsOffs

 HandsOffs is a class that represents a hands-off event in a game. It has the following attributes:
 - `id`: a unique identifier for the hands-off event
 - `player_id`: the ID of the player who performed the hands-off
 - `timestamp`: the timestamp of the hands-off event
 - `game_id`: the ID of the game where the hands-off occurred
 - `game_state`: the state of the game at the time of the hands-off event
 - `hands_off_type`: the type of hands-off event (e.g. "hands_off",
 "hands_off_by_opponent", etc.)
 - `hands_off_reason`: the reason for the hands-off event (e.g. "opponent's move",
 "player's move", etc.)
 - `hands_off_target`: the target of the hands-off event (e.g. "player",
 "opponent", etc.)
 - `hands_off_target_id`: the ID of the target of the hands-off event
 - `hands_off_target_type`: the type of the target of the hands-off event (e.g . "player", "opponent", etc.)
 - `hands_off_target_name`: the name of the target of the hands-off event 
 - `hands_off_target_position`: the position of the target of the hands-off event
 - `hands_off_target_position_x`: the x-coordinate of the target of the hands-off event
 - `hands_off_target_position_y`: the y-coordinate of the target of the hands-off event
 - `hands_off_target_position_z`: the z-coordinate of the target of the hands-off event
 - `hands_off_target_position_x_min`: the minimum x-coordinate of the target of the hands-off
 event
 - `hands_off_target_position_x_max`: the maximum x-coordinate of the target of the hands-off
 event
 - `hands_off_target_position_y_min`: the minimum y-coordinate of the target of the hands-off
 event
 - `hands_off_target_position_y_max`: the maximum y-coordinate of the target of the hands-off
 event
 - `hands_off_target_position_z_min`: the minimum z-coordinate of the target of the hands-off
 event
 - `hands_off_target_position_z_max`: the maximum z-coordinate of the target of the hands-off
 event
 - `hands_off_target_position_x_avg`: the average x-coordinate of the target of the hands-off
 event
 - `hands_off_target_position_y_avg`: the average y-coordinate of the target of the hands-off
 event
 - `hands_off_target_position_z_avg`: the average z-coordinate of the target of the hands
 off event
 - `hands_off_target_position_x_std`: the standard deviation of the x-coordinate of the target of
 the hands-off event
 - `hands_off_target_position_y_std`: the standard deviation of the y-coordinate of the target of
 the hands-off event
 - `hands_off_target_position_z_std`: the standard deviation of the z-coordinate of the target of
 the hands-off event
 - `hands_off_target_position_x_var`: the variance of the x-coordinate of the target of th
 hands-off event
 - `hands_off_target_position_y_var`: the variance of the y-coordinate of the target of th
 hands-off event
 - `hands_off_target_position_z_var`: the variance of the z-coordinate of the target of th
 hands-off event


In [40]:
from agents import Agent, Runner

tutorial_generator = Agent(
    name="Tutorial Generator",
    handoff_description= "Specialist agent for Openai sdk agent building tutorials",
    instructions=(
        "Given a programing topic and an outline, your job is to generate code snippets for each section of the outline."
        "format the tutorial in markdowns using a mix of text for explanation and code snippts for examples."
        "where it makes sense, include comments in the code snippts to further explain the code.",
    )

)

outline_builder = Agent(
    name= "Outline Builder",
    handoff_description= " Specialist agent for handoff",
    instructions={
        "diven a particular programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline."
        "after making the outline, hand it to the tutorial generator agent."
    },
    handoffs=[tutorial_generator]
    )
tutorial_response = await Runner.run(outline_builder, "What is handoff in openai agents")
print(tutorial_response.final_output)





Instructions must be a string or a function, got {'diven a particular programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline.after making the outline, hand it to the tutorial generator agent.'}


In the context of OpenAI agents, "handoff" refers to transferring control or a task from one agent to another specialized agent. This is used to leverage the expertise of different agents to handle various aspects of a task. For instance, if a request is made that requires specific knowledge or capabilities, the original agent may hand off the request to a more specialized agent to provide a more accurate or detailed response.


In [53]:
from agents import function_tool, Agent, handoff, Runner
from pydantic import BaseModel
import nest_asyncio
nest_asyncio.apply()

# Step 1: Define escalation input model
class ManagerEscalation(BaseModel):
    issue: str
    why: str

# Step 2: Define tool
@function_tool
def create_ticket(issues: str):
    print(f"Creating ticket for issue: {issues}")
    return "Ticket created ID: 12345"

# Step 3: Manager Agent
manager_agent = Agent(
    name="Manager Agent",
    handoff_description="Handles escalated issues that require managerial attention.",
    instructions=(
        "You handle escalated customer issues that the initial customer service agent could not solve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the customer, "
        "create a ticket for the issue to be solved by the system admin and inform the customer."
    ),
    tools=[create_ticket],
)

# Step 4: Optional handoff callback
def on_manager_handoff(ctx, input: ManagerEscalation):
    print("Escalating to manager agent:", input.issue)
    print("Reason for escalation:", input.why)

# Step 5: Customer Service Agent
customer_service_agent = Agent(
    name="Customer Service Agent",
    instructions=(
        "You assist customers with general inquiries and basic troubleshooting. "
        "If the issue cannot be resolved, you will escalate it to the manager along with the reason why you cannot fix the issue yourself."
    ),
    handoffs=[
        handoff(
            agent=manager_agent,
            input_type=ManagerEscalation,
            on_handoff=on_manager_handoff,
        )
    ]
)

# Step 6: Main async call
async def main():
    result = await Runner.run(
        customer_service_agent,
        "I want a refund , but your system won't let me process it. the website is just blank for me."
    )
    print(result.final_output)

# Step 7: Call it properly in Jupyter
await main()


Escalating to manager agent: Customer is unable to process a refund request online because the website displays as blank.
Reason for escalation: The issue involves a potentially serious website malfunction that requires technical expertise beyond basic troubleshooting.
Creating ticket for issue: Customer is unable to process a refund request online because the website displays as blank. Requires investigation of the website malfunction.
I apologize for the inconvenience. A ticket has been created to address this issue with our technical team. Your ticket ID is 12345, and we will resolve this as soon as possible. Thank you for your patience.


# Tracing

a way to see what the code is doing at each step what agents are doing at each step.This is useful for debugging and understanding the behavior of the code.

In [3]:
from agents import function_tool, Agent, handoff, Runner, trace  # ✅ now trace is imported
from pydantic import BaseModel
import nest_asyncio
nest_asyncio.apply()

class ManagerEscalation(BaseModel):
    issue: str
    why: str

@function_tool
def create_ticket(issues: str):
    print(f"Creating ticket for issue: {issues}")
    return "Ticket created ID: 12345"

manager_agent = Agent(
    name="Manager Agent",
    handoff_description="Handles escalated issues that require managerial attention.",
    instructions=(
        "You handle escalated customer issues that the initial customer service agent could not solve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the customer, "
        "create a ticket for the issue to be solved by the system admin and inform the customer."
    ),
    tools=[create_ticket],
)

def on_manager_handoff(ctx, input: ManagerEscalation):
    print("Escalating to manager agent:", input.issue)
    print("Reason for escalation:", input.why)

customer_service_agent = Agent(
    name="Customer Service Agent",
    instructions=(
        "You assist customers with general inquiries and basic troubleshooting. "
        "If the issue cannot be resolved, you will escalate it to the manager along with the reason why you cannot fix the issue yourself."
    ),
    handoffs=[
        handoff(
            agent=manager_agent,
            input_type=ManagerEscalation,
            on_handoff=on_manager_handoff,
        )
    ]
)

# ✅ Use trace in async function
async def main():
    with trace("Customer Service Hotline"):
        result = await Runner.run(
            customer_service_agent,
            "I want a refund , but your system won't let me process it. the website is just blank for me."
        )
        print(result.final_output)

# ✅ Jupyter-friendly execution
await main()


Escalating to manager agent: Customer is unable to process a refund on the website as it appears blank.
Reason for escalation: Technical troubleshooting of website issues is beyond my support capability.
Creating ticket for issue: Customer is unable to process a refund on the website as it appears blank. Requires technical investigation to resolve website display issue for refund processing.
I've created a ticket for your issue to be resolved by our technical team (Ticket ID: 12345). They will investigate the website problem and assist you with the refund as soon as possible. Thank you for your patience.


# Streaming

streaming lets you create a stream of data from a file or other source. This can be useful for large files or for processing data in real-time.


In [9]:
import random
from agents import Agent, ItemHelpers, Runner, function_tool
import nest_asyncio
nest_asyncio.apply()

# Tool: How many jokes
@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 5)

# Agent: Joker
agent = Agent(
    name="joker",
    instructions="First call the 'how_many_jokes' tool, then tell that many jokes.",
    tools=[how_many_jokes],
)

# Run agent stream
async def run_joker():
    print("=== Run starting ===")

   
    result = Runner.run_streamed(agent, input="Hello")

    # Now stream the events
    async for event in result.stream_events():
        if event.type == "raw_response_event":
            continue
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n{ItemHelpers.text_message_output(event.item)}")
    print("=== Run complete ===")

# ✅ Run it in notebook
await run_joker()


=== Run starting ===
Agent updated: joker
-- Tool was called
-- Tool output: 2
-- Message output:
Here are two jokes for you:

1. **Why did the scarecrow win an award?**
   Because he was outstanding in his field!

2. **Why don't scientists trust atoms?**
   Because they make up everything!
=== Run complete ===


In [7]:
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)

agent = Agent(
    name="Joker",
    instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
    tools=[how_many_jokes],
)

result = Runner.run_streamed(
    agent,
    input="Hello",
)
print("=== Run starting ===")

async for event in result.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue
    # When the agent updates, print that
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # When items are generated, print them
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types

print("=== Run complete ===")

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 2
-- Message output:
 Here are two jokes for you:

1. Why did the scarecrow win an award?  
   Because he was outstanding in his field!

2. What do you call fake spaghetti?  
   An impasta!
=== Run complete ===
